In [16]:
### Preprocessing the Yelp Comments on Berlin Restaurants

In [17]:
#### Get required packages

In [18]:
import pandas as pd
import os
import re
import nltk
import string
#from nltk import word_tokenize
#nltk.download()

#### Get dataset 

In [19]:
yelp = pd.read_csv('german_merged.csv')
yelp.columns.values[0]="ID"
yelp = yelp.rename(columns={'Overall Rating':'Overall_Rating',
                                      "Total Reviews":'Total_Reviews',
                                      "Restaurant Name":"Restaurant_Name",
                                     "Price Range":"Price_Range"})
yelp.columns
print(yelp['Comment'][1])

Das Joseph ist ein ziemlich neues, israelisches Restaurant im modernen Design.  Es ist zwar nicht besonders kuschelig,  aber angenehm hell und hübsch eingerichtet mit schönen Lampen.  Der Service ist ausgesprochen freundlich, unkompliziert und aufmerksam, besonders Celina. Das Essen kam schnell,  sah sehr appetitlich aus und so schmeckte es auch. Die israelischen Starter,  Kleinigkeiten und Hauptspeisen haben uns sehr gut geschmeckt,  die Preise sind angemessen und für Berlin-Mitte nicht zu teuer. Wir waren nur etwas überrascht,  dass schon um 21.30 / 22h sehr laute Musik aufgelegt wurde, wir konnten uns fast nicht mehr unterhalten am 2er Tisch gegenüber.  Auf Anfrage wurde das etwas gedrosselt. Fazit: Wer israelische Küche mag, gutes Preis-Leistungs-Verhältnis möchte mit Blick auf die Friedrichstraße und netten Service ist hier sehr gut aufgehoben.


In [20]:
yelp.dtypes

ID                   int64
Restaurant_Name     object
Overall_Rating     float64
Total_Reviews        int64
Specialty           object
Region              object
Price_Range         object
Author              object
Comment             object
Rating               int64
Date                object
dtype: object

In [21]:
yelp['Comment'] = yelp['Comment'].astype(str)
yelp['Overall_Rating'] = yelp['Overall_Rating'].astype(float)

yelp.Overall_Rating.head()

0    4.5
1    4.5
2    4.5
3    4.5
4    4.5
Name: Overall_Rating, dtype: float64

In [22]:
yelp.head()

,ID,Restaurant_Name,Overall_Rating,Total_Reviews,Specialty,Region,Price_Range,Author,Comment,Rating,Date
0,0,JOSEPH,4.5,6,Israelisch /,Mitte,Unknown,Dominik L.,Ich habe mir Bewertungen zu Restaurants auf Me...,5,26.10.2022
1,1,JOSEPH,4.5,6,Israelisch /,Mitte,Unknown,Regina D.,"Das Joseph ist ein ziemlich neues, israelische...",4,7.11.2019
2,2,JOSEPH,4.5,6,Israelisch /,Mitte,Unknown,Heiko B.,"Schalömchen, scheiße haben wir uns heute kosch...",5,18.10.2019
3,3,JOSEPH,4.5,6,Israelisch /,Mitte,Unknown,Torben B.,"Essen war allenfalls ok, muss ich nicht nochma...",2,8.1.2020
4,4,JOSEPH,4.5,6,Israelisch /,Mitte,Unknown,Conny J.,Waren auf Empfehlung dort und wurden nicht ent...,5,3.9.2019


In [23]:
yelp.Comment.values[0]

'Ich habe mir Bewertungen zu Restaurants auf Menulist angesehen, und dieses Restaurant hatte gute Kritiken, also habe ich es ausprobiert und es nicht bereut.menulist.menu/restaurant…'

#### Basic Cleaning: Remove punctuation and set everything to lower case

In [24]:
def clean_text(text):
    #Remove unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    #Filter to allow only alphabets
    #text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    
    #Fix &
    text = re.sub(r'&amp;', '&', text)
    
    #Remove punctuations etc.
    #text = re.sub(r'[?!.;:",#@-]', '', text)
    text = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', '', text)
    
    #Remove Prices and numbers
    text = re.sub(r'[0-9]+€|[0-9]+', '', text)

    #Convert to lowercase to maintain consistency
    text = text.lower()
    return text
yelp['Comment'] = yelp.Comment.apply(lambda x: clean_text(x))
#remove pattern "x Fotos" in Author - replace with Unknown
#remove comments in english
#

In [25]:
yelp.Comment.values[0]

'ich habe mir bewertungen zu restaurants auf menulist angesehen, und dieses restaurant hatte gute kritiken, also habe ich es ausprobiert und es nicht bereut.menulist.menu/restaurant'

#### Tokenization

In [26]:
# using re
# Explicitly tell Python that lower and upper case symbols are to be treated as the same symbol
def tokenize(text):
    tokens = re.split('\W+',text)
    return tokens
yelp['Comment_tokenized'] = yelp['Comment'].apply (lambda x: tokenize(x.lower()))

print(yelp.Comment_tokenized[0])

['ich', 'habe', 'mir', 'bewertungen', 'zu', 'restaurants', 'auf', 'menulist', 'angesehen', 'und', 'dieses', 'restaurant', 'hatte', 'gute', 'kritiken', 'also', 'habe', 'ich', 'es', 'ausprobiert', 'und', 'es', 'nicht', 'bereut', 'menulist', 'menu', 'restaurant']


#### Lemmatizing

In [27]:
wn = nltk.WordNetLemmatizer()
#dir(wn)

In [28]:
"""def lemmatizing(tokenized_text):
    text =  [wn.lemmatize(word) for word in tokenized_text]
    return text

yelp['Comment_lemmatized'] = yelp['Comment_tokenized'].apply(lambda x: lemmatizing(x))
"""

"def lemmatizing(tokenized_text):\n    text =  [wn.lemmatize(word) for word in tokenized_text]\n    return text\n\nyelp['Comment_lemmatized'] = yelp['Comment_tokenized'].apply(lambda x: lemmatizing(x))\n"

### Remove Punctuation, Tokenize, remove stopwords and lemmatize all in one

In [29]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = " ".join([wn.lemmatize(word) for word in tokens if word not in stopwords])
    return text

In [30]:
yelp['Comment_clean'] = yelp['Comment'].apply(lambda x: clean_text(x))

NameError: name 'stopwords' is not defined

In [ ]:
yelp[["ID","Comment_clean"]].to_csv("Yelp_Cleaned_Comments.csv",header=True)

### Vecotrization- Process of encoding text as integers to create feature vectors, basically transform char to numeric

#### 1. Count Vectorization

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(analyzer=clean_text)
X_counts = count_vect.fit_transform(yelp['Comment'])

# Peak into the words
print(X_counts.shape)
print(count_vect.get_feature_names()[189:204])

In [ ]:
X_counts

In [ ]:
X_counts_df = pd.DataFrame(X_counts.toarray())
X_counts_df

In [ ]:
X_counts_df.columns = count_vect.get_feature_names()
X_counts_df

#### N - Grams

In [ ]:
def clean_text_ngram(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = " ".join([wn.lemmatize(word) for word in tokens if word not in stopwords])
    return text

In [ ]:
yelp['Comment_clean' ] = yelp['Comment'].apply(lambda x : clean_text_ngram(x))
yelp.Comment_clean.head()

In [ ]:
ngram_vect = CountVectorizer(ngram_range=(2,2))
X_counts = ngram_vect.fit_transform(yelp['Comment_clean'])
print(X_counts.shape)
print(ngram_vect.get_feature_names())

In [ ]:
X_counts_df = pd.DataFrame(X_counts.toarray())
X_counts_df.columns = ngram_vect.get_feature_names()
X_counts_df

#### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(yelp['Comment'])
print(X_tfidf.shape)
print(tfidf_vect.get_feature_names()[1:15])

In [ ]:
X_tfidf_df = pd.DataFrame(X_tfidf.toarray())
X_tfidf_df.columns = tfidf_vect.get_feature_names_out()
X_tfidf_df[['service','food','experience','place']]
len(X_tfidf_df[X_tfidf_df['experience']==0])

### Feature Engineering 

In [ ]:
"""
Create new Features to best estimate the actual star rating given 
e.g. sentiment polarity & subjectivity with TextBlob,
length of text field, 
percentage of characters that are punctuation in the text. 
percentage of characters that are capitalized
"""
from textblob import TextBlob

In [ ]:
#dir(TextBlob)
help(TextBlob.sentiment_assessments)

In [ ]:
cmo = yelp['Comment_clean']

def senti_score(text):
    sentiment = []
    sentiment_temp = []
    result = []

    for elem in text:
        blob = TextBlob(elem)
        for sentence in blob.sentences:
            sentiment_temp.append(sentence.sentiment.polarity)
        sentiment = Average(sentiment_temp)
        result.append(sentiment)

yelp['Sentiment'] = result
yelp.head()

In [ ]:
# hypothesis: longer comments tend to be correlated with more positive overall rating


### EDA - Word frequencies

#### Removing Stop Words from Word Frequency Counts

In [ ]:
#pip install wordcloud

# Import list of stopwards
from wordcloud import STOPWORDS
print(STOPWORDS)

#### Generate frequency of every word from all comments

In [ ]:
def gen_freq(text):
    #Will store the list of words
    word_list = []

    #Loop over all the comments and extract words into word_list
    for tw_words in text.split():
        word_list.extend(tw_words)

    #Create word frequencies using word_list
    word_freq = pd.Series(word_list).value_counts()

    #Print top 5 and bottom 5 words
    word_freq[:10]
    
    return word_freq
gen_freq(yelp.Comment.str)

#### --> Problem: stopwords have highest frequencies 

In [ ]:
# Drop Stopwords from frequency table
word_freq = gen_freq(yelp.Comment.str)

word_freq = word_freq.drop(labels=STOPWORDS, errors='ignore')
word_freq.head()

In [ ]:
word_freq[156:164]

In [ ]:
# Import libraries for Plotting
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#Generate word cloud
wc = WordCloud(width=400, height=330, max_words=100, background_color='white').generate_from_frequencies(word_freq)

plt.figure(figsize=(12, 8))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()

### Basic ML application

##### Feature Engineering

In [ ]:
# Generate even greater list with stopwords 
stop_words_manual =['a', 'about', 'above', 'after', 'again', 'against', 'all', 'also', 'am', 'an', 'and',
              'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below',
              'between', 'both', 'but', 'by', 'can', "can't", 'cannot', 'com', 'could', "couldn't", 'did',
              "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'else', 'ever',
              'few', 'for', 'from', 'further', 'get', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having',
              'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how',
              "how's", 'however', 'http', 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it',
              "it's", 'its', 'itself', 'just', 'k', "let's", 'like', 'me', 'more', 'most', "mustn't", 'my', 'myself',
              'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'otherwise', 'ought', 'our', 'ours',
              'ourselves', 'out', 'over', 'own', 'r', 'same', 'shall', "shan't", 'she', "she'd", "she'll", "she's",
              'should', "shouldn't", 'since', 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs',
              'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", "they'll", "they're",
              "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', "wasn't",
              'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
              "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's", 'with', "won't", 'would', "wouldn't",
              'www', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']

# Merge Manual list and wordcloud's stopwords list
stop_words = list(STOPWORDS)+stop_words_manual

# but remove duplicates
stop_words = [*set(stop_words)]

print(stop_words)

In [ ]:
# Regenrate Frequency of all words
freq_clean = gen_freq(yelp.Comment.str).drop(labels=stop_words, errors='ignore')

# how many different words do we still have 
# code to count number of entries in freq_clean

# Get 100 rarest words only
rare_100 =freq_clean[-100:]
rare_100.head()

#### Create new variables per restaurant comment from characteristics of Comment - Feature Engineering

In [ ]:
#Check whether a negation term is present in the text
def any_neg(words):
    for word in words:
        if word in ['n', 'no', 'non', 'not'] or re.search(r"\wn't", word):
            return 1
    else:
        return 0
    
#Check whether one of the 100 rare words is present in the text
def any_rare(words, rare_100):
    for word in words:
        if word in rare_100:
            return 1
    else:
        return 0

#Check whether prompt words are present
def is_question(words):
    for word in words:
        if word in ['when', 'what', 'how', 'why', 'who']:
            return 1
    else:
        return 0

In [ ]:
#Number of words in a comment
yelp['word_count'] = yelp.Comment.str.split().apply(lambda x: len(x))
#Negation present or not
yelp['any_neg'] = yelp.Comment.str.split().apply(lambda x: any_neg(x))
#Prompt present or not
yelp['is_question'] = yelp.Comment.str.split().apply(lambda x: is_question(x))
#Any of the most 100 rare words present or not
yelp['any_rare'] = yelp.Comment.str.split().apply(lambda x: any_rare(x, rare_100))

In [ ]:
print(yelp.head())

#### Basic ML : Split dataset into predicting variables and predicted variable as well as into training dataset and test dataset

In [ ]:
from sklearn.model_selection import train_test_split

X = yelp[['word_count', 'any_neg', 'any_rare', 'Price_Range', 'is_question','Overall_Rating']]
y = yelp[['Rating']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=27)

##### 4. Train an ML model for Text Classification

##### Now that the dataset is ready, it is time to train a Machine Learning model on the same. You will be using a Naive Bayes classifier from sklearn which is a prominent python library used for machine learning.

In [ ]:
from sklearn.naive_bayes import GaussianNB

#Initialize GaussianNB classifier
model = GaussianNB()
#Fit the model on the train dataset
model = model.fit(X_train, y_train)
#Make predictions on the test datasetsyd
pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

print("Accuracy:", accuracy_score(y_test, pred)*100, "%")